In [1]:
from numpy.random import multinomial
from numpy import log, exp
from numpy import argmax
import json

class MovieGroupProcess:
    def __init__(self, K=8, alpha=0.1, beta=0.1, n_iters=30):
        '''
        A MovieGroupProcess is a conceptual model introduced by Yin and Wang 2014 to
        describe their Gibbs sampling algorithm for a Dirichlet Mixture Model for the
        clustering short text documents.
        Reference: http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf
        Imagine a professor is leading a film class. At the start of the class, the students
        are randomly assigned to K tables. Before class begins, the students make lists of
        their favorite films. The teacher reads the role n_iters times. When
        a student is called, the student must select a new table satisfying either:
            1) The new table has more students than the current table.
        OR
            2) The new table has students with similar lists of favorite movies.
        :param K: int
            Upper bound on the number of possible clusters. Typically many fewer
        :param alpha: float between 0 and 1
            Alpha controls the probability that a student will join a table that is currently empty
            When alpha is 0, no one will join an empty table.
        :param beta: float between 0 and 1
            Beta controls the student's affinity for other students with similar interests. A low beta means
            that students desire to sit with students of similar interests. A high beta means they are less
            concerned with affinity and are more influenced by the popularity of a table
        :param n_iters:
        '''
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.n_iters = n_iters

        # slots for computed variables
        self.number_docs = None
        self.vocab_size = None
        self.cluster_doc_count = [0 for _ in range(K)]
        self.cluster_word_count = [0 for _ in range(K)]
        self.cluster_word_distribution = [{} for i in range(K)]

    @staticmethod
    def from_data(K, alpha, beta, D, vocab_size, cluster_doc_count, cluster_word_count, cluster_word_distribution):
        '''
        Reconstitute a MovieGroupProcess from previously fit data
        :param K:
        :param alpha:
        :param beta:
        :param D:
        :param vocab_size:
        :param cluster_doc_count:
        :param cluster_word_count:
        :param cluster_word_distribution:
        :return:
        '''
        mgp = MovieGroupProcess(K, alpha, beta, n_iters=30)
        mgp.number_docs = D
        mgp.vocab_size = vocab_size
        mgp.cluster_doc_count = cluster_doc_count
        mgp.cluster_word_count = cluster_word_count
        mgp.cluster_word_distribution = cluster_word_distribution
        return mgp

    @staticmethod
    def _sample(p):
        '''
        Sample with probability vector p from a multinomial distribution
        :param p: list
            List of probabilities representing probability vector for the multinomial distribution
        :return: int
            index of randomly selected output
        '''
        return [i for i, entry in enumerate(multinomial(1, p)) if entry != 0][0]

    def fit(self, docs, vocab_size):
        '''
        Cluster the input documents
        :param docs: list of list
            list of lists containing the unique token set of each document
        :param V: total vocabulary size for each document
        :return: list of length len(doc)
            cluster label for each document
        '''
        alpha, beta, K, n_iters, V = self.alpha, self.beta, self.K, self.n_iters, vocab_size

        D = len(docs)
        self.number_docs = D
        self.vocab_size = vocab_size

        # unpack to easy var names
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution
        cluster_count = K
        d_z = [None for i in range(len(docs))]

        # initialize the clusters
        for i, doc in enumerate(docs):

            # choose a random  initial cluster for the doc
            z = self._sample([1.0 / K for _ in range(K)])
            d_z[i] = z
            m_z[z] += 1
            n_z[z] += len(doc)

            for word in doc:
                if word not in n_z_w[z]:
                    n_z_w[z][word] = 0
                n_z_w[z][word] += 1

        for _iter in range(n_iters):
            total_transfers = 0

            for i, doc in enumerate(docs):

                # remove the doc from it's current cluster
                z_old = d_z[i]

                m_z[z_old] -= 1
                n_z[z_old] -= len(doc)

                for word in doc:
                    n_z_w[z_old][word] -= 1

                    # compact dictionary to save space
                    if n_z_w[z_old][word] == 0:
                        del n_z_w[z_old][word]

                # draw sample from distribution to find new cluster
                p = self.score(doc)
                z_new = self._sample(p)

                # transfer doc to the new cluster
                if z_new != z_old:
                    total_transfers += 1

                d_z[i] = z_new
                m_z[z_new] += 1
                n_z[z_new] += len(doc)

                for word in doc:
                    if word not in n_z_w[z_new]:
                        n_z_w[z_new][word] = 0
                    n_z_w[z_new][word] += 1

            cluster_count_new = sum([1 for v in m_z if v > 0])
            print("In stage %d: transferred %d clusters with %d clusters populated" % (
            _iter, total_transfers, cluster_count_new))
            if total_transfers == 0 and cluster_count_new == cluster_count and _iter>25:
                print("Converged.  Breaking out.")
                break
            cluster_count = cluster_count_new
        self.cluster_word_distribution = n_z_w
        return d_z

    def score(self, doc):
        '''
        Score a document
        Implements formula (3) of Yin and Wang 2014.
        http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf
        :param doc: list[str]: The doc token stream
        :return: list[float]: A length K probability vector where each component represents
                              the probability of the document appearing in a particular cluster
        '''
        alpha, beta, K, V, D = self.alpha, self.beta, self.K, self.vocab_size, self.number_docs
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution

        p = [0 for _ in range(K)]

        #  We break the formula into the following pieces
        #  p = N1*N2/(D1*D2) = exp(lN1 - lD1 + lN2 - lD2)
        #  lN1 = log(m_z[z] + alpha)
        #  lN2 = log(D - 1 + K*alpha)
        #  lN2 = log(product(n_z_w[w] + beta)) = sum(log(n_z_w[w] + beta))
        #  lD2 = log(product(n_z[d] + V*beta + i -1)) = sum(log(n_z[d] + V*beta + i -1))

        lD1 = log(D - 1 + K * alpha)
        doc_size = len(doc)
        for label in range(K):
            lN1 = log(m_z[label] + alpha)
            lN2 = 0
            lD2 = 0
            for word in doc:
                lN2 += log(n_z_w[label].get(word, 0) + beta)
            for j in range(1, doc_size +1):
                lD2 += log(n_z[label] + V * beta + j - 1)
            p[label] = exp(lN1 - lD1 + lN2 - lD2)

        # normalize the probability vector
        pnorm = sum(p)
        pnorm = pnorm if pnorm>0 else 1
        return [pp/pnorm for pp in p]

    def choose_best_label(self, doc):
        '''
        Choose the highest probability label for the input document
        :param doc: list[str]: The doc token stream
        :return:
        '''
        p = self.score(doc)
        return argmax(p),max(p)

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np
import numpy 

class Classifier(object):

    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        X_train = [self.embeddings[int(x)] for x in X]
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y]
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro", "samples", "weighted"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        results['accuracy'] = accuracy_score(Y, Y_)
        
        # print('Results, using embeddings of dimensionality', len(self.embeddings[X[0]]))
        # print('-------------------')
        #print(results)
        return results
        # print('-------------------')

    def predict(self, X, top_k_list):
        X_ = numpy.asarray([self.embeddings[int(x)] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = numpy.random.get_state()

        training_size = int(train_precent * len(X))
        numpy.random.seed(seed)
        shuffle_indices = numpy.random.permutation(numpy.arange(len(X)))
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y)
        numpy.random.set_state(state)
        return self.evaluate(X_test, Y_test)


from sklearn.multiclass import OneVsRestClassifier
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = numpy.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return numpy.asarray(all_labels)
    

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time
import re
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score

filename = "Tweet"
#filename = "T"
#filename = "S"
#filename = "TS"
data_path = "./../data/"+filename
data = []
docs = []
total_doc_num = 2472
doc_num = 0
doc_label = np.zeros((total_doc_num,),dtype=int)
with open(data_path,'r')as file:
    punc = '",:'
    lines = file.readlines()
    for line in lines:
        line = re.sub(r'[{}]+'.format(punc),'',line)
        
        data.append(" ".join(line.split(' ')[1:-2]))
        doc_label[doc_num] = int(line.split(' ')[-1].strip().replace("}",""))
        docs.append(line.split(' ')[1:-2])
        
        doc_num += 1
file.close()

In [4]:
uni_set = set()
for count in data:
    for j in count.split(" "):
        uni_set.add(j)

In [5]:
from sklearn.utils.linear_assignment_ import linear_assignment
def acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

D:\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [6]:
n_cluster = 89
beta = 0.1
mgp = MovieGroupProcess(K=n_cluster, alpha=0.1, beta=beta, n_iters=10)
y = mgp.fit(docs,len(uni_set))

In stage 0: transferred 2213 clusters with 89 clusters populated
In stage 1: transferred 903 clusters with 86 clusters populated
In stage 2: transferred 422 clusters with 81 clusters populated
In stage 3: transferred 314 clusters with 76 clusters populated
In stage 4: transferred 260 clusters with 74 clusters populated
In stage 5: transferred 242 clusters with 73 clusters populated
In stage 6: transferred 223 clusters with 69 clusters populated
In stage 7: transferred 200 clusters with 68 clusters populated
In stage 8: transferred 154 clusters with 67 clusters populated
In stage 9: transferred 158 clusters with 67 clusters populated


In [7]:
mgp_doc_label = np.zeros((total_doc_num,),dtype=int)
for i in range(len(docs)):
    mgp_doc_label[i] = mgp.choose_best_label(docs[i])[0]

In [8]:
normalized_mutual_info_score(np.array(doc_label), np.array(mgp_doc_label))

D:\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.8545964687694341

In [9]:
filename = "./../Tweet_dictionary.txt"
word_id = {}
with open(filename,'r')as datafile:
    sents = datafile.readlines()
    for data in sents:
        word_id[data.split(" ")[0]] = int(data.split(" ")[1].strip())
datafile.close()

In [10]:
beta = 0.001
topic_word_dis = np.zeros((n_cluster,len(uni_set)))
for i in range(len(mgp.cluster_word_distribution)):
    for j,keyword in enumerate(word_id):
        if keyword not in mgp.cluster_word_distribution[i]:
            topic_word_dis[i][j] = beta
        else:
            topic_word_dis[i][j] = mgp.cluster_word_distribution[i][keyword] + beta

In [11]:
norm_topic_word_dis = topic_word_dis / np.sum(topic_word_dis, axis = 1)[:, np.newaxis]

In [12]:
import numpy as np
total_doc_num = 2472
doc_emb = np.zeros((total_doc_num,n_cluster))
doc_label = np.zeros((total_doc_num,),dtype=int)
import re
filename = "./../data/Tweet"
doc_num = 0
with open(filename,'r')as datafile:
        sents = datafile.readlines()
        punc = '",:'
        for data in sents:
            
            data = re.sub(r'[{}]+'.format(punc),'',data)
            raw_text = data.split(' ')[1:-2]
            
            doc_label[doc_num] = int(data.split(' ')[-1].strip().replace("}",""))
            for data_i in raw_text:
                doc_emb[doc_num] += norm_topic_word_dis[:,word_id[data_i]]
            doc_emb[doc_num] /= len(raw_text)  
            #doc_emb[doc_num]
            doc_num += 1
datafile.close()            

In [19]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=89,  max_iter=100).fit(doc_emb)

In [14]:
from sklearn.metrics.cluster import normalized_mutual_info_score

In [15]:
print(normalized_mutual_info_score(np.array(doc_label), np.array(kmeans.labels_)))
print(acc(np.array(doc_label), np.array(kmeans.labels_)))

0.8122011914659129
0.55542071197411


D:\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [21]:
import warnings
warnings.filterwarnings('ignore')

avg_nmi = []
avg_acc = []
for i in range(20):
    kmeans = KMeans(n_clusters=89,  max_iter=100).fit(doc_emb)
    avg_nmi.append(normalized_mutual_info_score(np.array(doc_label), np.array(kmeans.labels_)))
    avg_acc.append(acc(np.array(doc_label), np.array(kmeans.labels_)))
print("avg_nmi: ", np.mean(avg_nmi))
print("avg_acc: ",np.mean(avg_acc))

avg_nmi:  0.804737707149681
avg_acc:  0.5222694174757281


In [13]:
Y = [str(i) for i in list(doc_label)]
X = [str(i) for i in list(np.arange(total_doc_num))]

In [14]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=doc_emb, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['micro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['micro'] )
print(np.mean(avg_value))

0.1   0.2994788650839606
0.2   0.2984996738421396
0.3   0.31233283803863304
0.4   0.3583694709453599
0.5   0.3044838373305526
0.6   0.31134289439374185
0.7   0.3647140864714086
0.8   0.36649214659685864
0.9   0.4033970276008492
0.33545676003372266


In [15]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=doc_emb, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['macro'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['macro'] )
print(np.mean(avg_value))

0.1   0.13313127347008674
0.2   0.13315736843600212
0.3   0.1444896236650231
0.4   0.1625124967329073
0.5   0.13919554919649346
0.6   0.1469360587268253
0.7   0.18359720266543103
0.8   0.1958220438211641
0.9   0.20096162887981545
0.15997813839930541


In [16]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
avg_value = []
for i in np.arange(0.1, 1.0, 0.1):
#for i in np.arange(0.01, 0.11, 0.01):
    clf_ratio = np.round(i,2) 
    clf = Classifier(vectors=doc_emb, clf=LogisticRegression())
    avg_value.append(clf.split_train_evaluate(X, Y, clf_ratio)['accuracy'])
    print( clf_ratio," ",clf.split_train_evaluate(X, Y, clf_ratio)['accuracy'] )
print(np.mean(avg_value))

0.1   0.02696629213483146
0.2   0.03083923154701719
0.3   0.04737146158290006
0.4   0.1078167115902965
0.5   0.07605177993527508
0.6   0.07987866531850354
0.7   0.11185983827493262
0.8   0.09090909090909091
0.9   0.12903225806451613
0.07785836992859595


In [17]:
storefile = 'emb/Tweet_gsdmm_emb.txt'
sf = open(storefile,'w')
for i in range(doc_emb.shape[0]):
    sf.write(str(i)+" "+" ".join([str(ele) for ele in doc_emb[i]])+'\n')
sf.close()